In [4]:
import pandas as pd
label_proteingym_map = {
    'BG_STRSQ_Abate2015-enrichment':'Q59976_STRSQ_Romero_2015',
    'BLAT_ECOLX_Ranganathan2015-2500': 'BLAT_ECOLX_Stiffler_2015',
    'BLAT_ECOLX_Ostermeier2014-linear':'BLAT_ECOLX_Firnberg_2014',
    'BLAT_ECOLX_Palzkill2012-ddG_stat':'BLAT_ECOLX_Deng_2012',
    'BLAT_ECOLX_Tenaillon2013-singles-MIC_score':'BLAT_ECOLX_Jacquier_2013',
    'DLG4_RAT_Ranganathan2012-CRIPT':'DLG4_RAT_McLaughlin_2012', # pdz
    'GAL4_YEAST_Shendure2015-SEL_C_40h':'GAL4_YEAST_Kitzman_2015',
    'HSP82_YEAST_Bolon2016-selection_coefficient': 'HSP82_YEAST_Mishra_2016',
    'KKA2_KLEPN_Mikkelsen2014-Kan18_avg':'KKA2_KLEPN_Melnikov_2014',
    'RL401_YEAST_Bolon2013-selection_coefficient':'RL40A_YEAST_Roscoe_2013',
    'RL401_YEAST_Bolon2014-react_rel':'RL40A_YEAST_Roscoe_2014',
    'YAP1_HUMAN_Fields2012-singles-linear':'YAP1_HUMAN_Araya_2012'
    
    
    
}

    # 'BRCA1_HUMAN_Fields2015-e3': ?
    # 'GFP_AEQVI_Sarkisyan2016': 'GFP_AEQVI_Sarkisyan_2016'
    # 'MTH3_HAEAESTABILIZED_Tawfik2015-Wrel_G17_filtered': 'MTH3_HAEAE_RockahShmuel_2015'
    #''PABP_YEAST_Fields2013-linear':'PABP_YEAST_Melamed_2013'
    # 'POLG_HCVJF_Sun2014-fitness':'POLG_HCVJF_Qi_2014'
    # 'UBE4B_MOUSE_Klevit2013-nscor_log2_ratio':'UBE4B_MOUSE_Starita_2013'

dataset_score = {}
for label_name,proteingym_name in label_proteingym_map.items():
    proteingym_ground_truth_file = f'/work/commons/proteingym/DMS_ProteinGym_substitutions/{proteingym_name}.csv'
    
    models = ['ESM1b','EVmutation','DeepSequence','UniRep_evotuned']
    value_name = ['esm1b_t33_650M_UR50S','prediction_independent','evol_indices_ensemble','Unirep_score']
    value_name = dict(zip(models,value_name))
    models_dict = {}
    for model in models:
        proteingym_model_file = f'/work/commons/proteingym/zero_shot_substitutions_scores/{model}/{proteingym_name}.csv'
        proteingym_model = pd.read_csv(proteingym_model_file)
        models_dict[model] = proteingym_model

    label_file = f'/nethome/zli3161/DATA-nash/combining-evolutionary-and-assay-labelled-data/data/learning-protein-fitness-from-evolutionary-and-assay-labelled-data/processed_data/{label_name}/data.csv'
    import pandas as pd
    proteingym_gt_file = pd.read_csv(proteingym_ground_truth_file)
    label = pd.read_csv(label_file)
    scores = dict(zip(models,[[] for i in range(len(models))]))
    for i, row in label.iterrows():
        
        mutant_key = 'mutant'
        # print(row[mutant_key])
        if row[mutant_key] == 'WT':
            continue

        if label_name in ['DLG4_RAT_Ranganathan2012-CRIPT'] and row[mutant_key] not in proteingym_gt_file[mutant_key].values:
            print(row[mutant_key])
            continue
        assert row[mutant_key] in proteingym_gt_file[mutant_key].values
        idx = proteingym_gt_file[proteingym_gt_file[mutant_key] == row[mutant_key]].index[0]
        # check mutated sequence
        assert row[mutant_key] in proteingym_gt_file[mutant_key][idx]
        
        for model in models:
            # print(model)
            proteingym_model = models_dict[model]
            value = value_name[model]
            if model is not 'UniRep_evotuned':
                model_score = proteingym_model[proteingym_model[mutant_key] == row[mutant_key]][value]
            else:
                model_score = proteingym_model[proteingym_model['mutated_sequence'] == proteingym_gt_file['mutated_sequence'][idx]][value]
            scores[model].append(model_score.values[0])
    dataset_score[label_name] = scores

P311A


In [10]:
dataset_score['BLAT_ECOLX_Ranganathan2015-2500']['ESM1b']

4807

In [22]:
import pandas as pd
esm_result = pd.read_csv('/nethome/zli3161/DATA-nash/combining-evolutionary-and-assay-labelled-data/inference/BLAT_ECOLX_Ranganathan2015-2500/esm/global/pll_org.csv',index_col=0)

In [23]:
esm_result['pll'] = dataset_score['BLAT_ECOLX_Ranganathan2015-2500']['ESM1b']

In [24]:
# save csv
esm_result.to_csv('/nethome/zli3161/DATA-nash/combining-evolutionary-and-assay-labelled-data/inference/BLAT_ECOLX_Ranganathan2015-2500/esm/global/pll.csv',index=True)

In [12]:
from scipy.stats import spearmanr
spearmanr(dataset_score['BLAT_ECOLX_Ranganathan2015-2500']['ESM1b'],esm_result['pll'])

SpearmanrResult(correlation=0.9692026602547521, pvalue=0.0)

In [32]:
pd.read_csv('/nethome/zli3161/DATA-nash/combining-evolutionary-and-assay-labelled-data/results/BLAT_ECOLX_Ranganathan2015-2500/results.csv').spearman.mean()

0.84107